In [ ]:
!pip install torch torchvision numba pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import glob
import os
from numba import jit, prange

# Paths
train_folder = "/content/drive/My Drive/Colab Notebooks/train_images"
test_folder = "/content/drive/My Drive/Colab Notebooks/test_images"
single_test_image = "/content/drive/My Drive/Colab Notebooks/test_images/200003.jpg"

# CUDA-accelerated CPU normalization with Numba
@jit(nopython=True, parallel=True)
def normalize_images(images):
    norm_images = np.empty_like(images, dtype=np.float32)
    for i in prange(images.shape[0]):
        for j in range(images.shape[1]):
            for k in range(images.shape[2]):
                for c in range(images.shape[3]):
                    norm_images[i, j, k, c] = images[i, j, k, c] / 255.0
    return norm_images

# Load and preprocess image dataset
def load_and_preprocess_images(folder_path, image_size=(128, 128), limit=None):
    image_paths = []
    labels = []
    class_to_idx = {}

    for idx, class_name in enumerate(sorted(os.listdir(folder_path))):
        class_path = os.path.join(folder_path, class_name)
        if not os.path.isdir(class_path):
            continue
        class_to_idx[class_name] = idx
        for img_file in glob.glob(class_path + '/*.jpg'):
            image_paths.append(img_file)
            labels.append(idx)
            if limit and len(image_paths) >= limit:
                break

    num_images = len(image_paths)
    img_array = np.zeros((num_images, image_size[0], image_size[1], 3), dtype=np.uint8)

    for i, img_path in enumerate(image_paths):
        img = Image.open(img_path).resize(image_size).convert('RGB')
        img_array[i] = np.array(img)

    img_array = normalize_images(img_array)
    img_array = np.transpose(img_array, (0, 3, 1, 2))  # NHWC ➜ NCHW
    return torch.from_numpy(img_array), torch.tensor(labels), class_to_idx

# CNN Model
class PaddyCNN(nn.Module):
    def __init__(self, num_classes):
        super(PaddyCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# Load training data
X_train, y_train, class_to_idx = load_and_preprocess_images(train_folder)
num_classes = len(class_to_idx)
print(" Classes:", class_to_idx)

# Dataset and Dataloader
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.float()
        self.y = y.long()
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(" Using device:", device)

# Initialize model
model = PaddyCNN(num_classes).to(device)
print("Model on GPU:", next(model.parameters()).is_cuda)

# Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f" Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

# Index mapping
idx_to_class = {v: k for k, v in class_to_idx.items()}

# Predict all images in test folder
def predict_test_images(test_folder_path):
    model.eval()
    image_files = glob.glob(test_folder_path + '/*.jpg')
    for img_file in image_files:
        img = Image.open(img_file).resize((128, 128)).convert('RGB')
        img_np = np.array(img).reshape(1, 128, 128, 3)
        norm_img = normalize_images(img_np)
        norm_img = np.transpose(norm_img, (0, 3, 1, 2))
        tensor_img = torch.from_numpy(norm_img).float().to(device)

        with torch.no_grad():
            output = model(tensor_img)
            _, predicted = torch.max(output, 1)
            label = idx_to_class[predicted.item()]
            print(f" {os.path.basename(img_file)} ➜ {label}")



 Classes: {'bacterial_leaf_blight': 0, 'bacterial_leaf_streak': 1, 'bacterial_panicle_blight': 2, 'blast': 3, 'brown_spot': 4, 'dead_heart': 5, 'downy_mildew': 6, 'hispa': 7, 'normal': 8, 'tungro': 9}
 Using device: cuda
Model on GPU: True
 Epoch 1/10, Loss: 213.0809
 Epoch 2/10, Loss: 159.0382
 Epoch 3/10, Loss: 122.1100
 Epoch 4/10, Loss: 90.6016
 Epoch 5/10, Loss: 65.5104
 Epoch 6/10, Loss: 49.5500
 Epoch 7/10, Loss: 36.9395
 Epoch 8/10, Loss: 31.2787
 Epoch 9/10, Loss: 22.3333
 Epoch 10/10, Loss: 18.6259


In [ ]:
def predict_single_image(img_path):
    model.eval()
    img = Image.open(img_path).resize((128, 128)).convert('RGB')
    img_np = np.array(img).reshape(1, 128, 128, 3)
    norm_img = normalize_images(img_np)
    norm_img = np.transpose(norm_img, (0, 3, 1, 2))
    tensor_img = torch.from_numpy(norm_img).float().to(device)

    with torch.no_grad():
        output = model(tensor_img)
        _, predicted = torch.max(output, 1)
        label = idx_to_class[predicted.item()]
        print(f" Single Prediction : {label}")
predict_single_image(single_test_image)



 Single Prediction ➜ normal


In [ ]:
import torch
torch.save(model.state_dict(), "paddy_model.pth")

In [ ]:
torch.save(model.state_dict(), "/content/drive/My Drive/paddy_model.pth")

In [ ]:
import pickle

with open("class_to_idx.pkl", "wb") as f:
    pickle.dump(class_to_idx, f)


In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


In [ ]:
%%writefile paddyapp.py
import streamlit as st
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import pickle

# Load class mapping
with open("class_to_idx.pkl", "rb") as f:
    class_to_idx = pickle.load(f)
idx_to_class = {v: k for k, v in class_to_idx.items()}

# Model definition (same as training)
class PaddyCNN(nn.Module):
    def __init__(self, num_classes):
        super(PaddyCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PaddyCNN(len(class_to_idx))
model.load_state_dict(torch.load("paddy_model.pth", map_location=device))
model.eval()

# Normalize function
def normalize_image(img_array):
    img_array = img_array.astype(np.float32) / 255.0
    img_array = np.transpose(img_array, (2, 0, 1))  # HWC ➜ CHW
    return torch.tensor(img_array).unsqueeze(0)

# Streamlit UI
st.title(" Paddy Leaf Disease Detector")
st.write("Upload a leaf image to predict the disease.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])
if uploaded_file is not None:
    image = Image.open(uploaded_file).resize((128, 128)).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Prediction
    img_array = np.array(image)
    tensor_img = normalize_image(img_array).to(device)

    with torch.no_grad():
        output = model(tensor_img)
        _, predicted = torch.max(output, 1)
        label = idx_to_class[predicted.item()]
        st.success(f" Predicted Disease: **{label}**")


Overwriting paddyapp.py


In [ ]:
%%writefile paddyapp.py
import streamlit as st
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import pickle

# Load class mapping
with open("class_to_idx.pkl", "rb") as f:
    class_to_idx = pickle.load(f)
idx_to_class = {v: k for k, v in class_to_idx.items()}

# Model definition (same as training)
class PaddyCNN(nn.Module):
    def __init__(self, num_classes):
        super(PaddyCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PaddyCNN(len(class_to_idx))
model.load_state_dict(torch.load("paddy_model.pth", map_location=device))
model.eval()

# Normalize function
def normalize_image(img_array):
    img_array = img_array.astype(np.float32) / 255.0
    img_array = np.transpose(img_array, (2, 0, 1))  # HWC ➜ CHW
    return torch.tensor(img_array).unsqueeze(0)

# Streamlit UI
st.title(" Paddy Leaf Disease Detector")
st.write("Upload a leaf image to predict the disease.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])
if uploaded_file is not None:
    image = Image.open(uploaded_file).resize((128, 128)).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Prediction
    img_array = np.array(image)
    tensor_img = normalize_image(img_array).to(device)

    with torch.no_grad():
        output = model(tensor_img)
        _, predicted = torch.max(output, 1)
        label = idx_to_class[predicted.item()]
        st.success(f" Predicted Disease: **{label}**")

Overwriting paddyapp.py
